In [28]:
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
import pandas as pd
import numpy as np
%matplotlib inline

In [2]:
tci = pd.read_csv('model_data/tci_2_2.csv')
ppns = set(tci['parcel'])
ppns_num = set(tci['parcel'].apply(lambda x: x.replace('-','')))

## Foreclosure data

In [46]:
fc = pd.read_csv('clean_data/foreclosure_filings_tci.csv', parse_dates=[2])

In [47]:
fc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5843 entries, 0 to 5842
Data columns (total 26 columns):
parcel            5843 non-null object
status            5843 non-null object
filedate          5843 non-null datetime64[ns]
defendant         5843 non-null object
caseno            5843 non-null object
parcel_address    5823 non-null object
dateid            5843 non-null object
FORE              5843 non-null int64
LATITUDE          5821 non-null float64
LONGITUDE         5821 non-null float64
zip_fore          5676 non-null float64
case_number       5843 non-null object
ctitle            5843 non-null object
cdesig            5843 non-null object
judge             5843 non-null object
magistrate        5143 non-null object
room              17 non-null object
n_action          17 non-null object
f_location        5843 non-null object
lstatus           5843 non-null object
lstatusdt         5843 non-null object
ldispos           5843 non-null object
ldisposdt         5843 non-nu

In [48]:
fc.head(2)

,parcel,status,filedate,defendant,caseno,parcel_address,dateid,FORE,LATITUDE,LONGITUDE,...,room,n_action,f_location,lstatus,lstatusdt,ldispos,ldisposdt,pamount,detaildt,plaintiff
0,121-33-110,Inactive,2006-10-03,TRACY M. THOMPSON,BR-06-000022,11028 MT CARMEL RD,03/21/2015,1,41.489396,-81.608232,...,NaN,NaN,DF-ROOM 45,INACTIVE,01/23/2007,DEFAULT,01/17/2007,$.00,03/21/2015,"JAMES ROKAKIS, AS TREASURER"
1,130-11-109,Inactive,2006-10-03,ALLEN ENTERPRISES IV LLC,BR-06-000028,3294 EAST 140TH STREET,03/21/2015,1,41.469930,-81.585067,...,NaN,NaN,DF-ROOM 45,INACTIVE,03/16/2007,DEFAULT,03/13/2007,$.00,12/24/2014,"JAMES ROKAKIS, AS TREASURER"


In [49]:
fc[fc.lstatus=='ACTIVE'].shape

(91, 26)

In [136]:
def parse_amount(x):
    return float(x[1:].replace(',',''))

In [72]:
fc['pamount'] = fc['pamount'].apply(parse_amount)

IndexError: invalid index to scalar variable.

In [105]:
tem = fc.loc[fc.filedate>dt.datetime(2011, 3, 1),['parcel','pamount']].groupby('parcel').first()
tem.columns = ['foreclosure_flag']

tci = pd.merge(tci, tem, \
             how='left', left_on='parcel', right_index=True)

In [108]:
for dis in set(fc.ldispos):
    a = tci[tci.parcel.isin(set(fc[fc.ldispos==dis].parcel))]
    print dis, len(a), sum(a.vacant)*1.0/len(a.vacant)

DIS. W/PREJ 54 0.240740740741
ADR 5 0.0
NEWLY FILED 46 0.195652173913
DISP.OTHER 1111 0.239423942394
DEFAULT 2202 0.217983651226
DISPOSED 1 0.0
BANKRPT/C.O.A. STAY 32 0.125
DIS. W/O PREJ 1832 0.223253275109
MAGISTRATE 38 0.210526315789
DISP.COURT TRIAL 1 1.0


## Sheriff's auction

In [109]:
sa = pd.read_csv('clean_data/shr_auction_tci.csv', parse_dates=[2])

In [111]:
sa.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5994 entries, 0 to 5993
Data columns (total 25 columns):
parcel        5994 non-null object
address       5977 non-null object
salesdt       5994 non-null datetime64[ns]
saleno        5994 non-null float64
sold_amt      3624 non-null object
appraisal     4601 non-null object
minbid        4605 non-null object
scaseno       5994 non-null object
withdrawn     2470 non-null object
plaintiff     5992 non-null object
sdefend       5994 non-null object
purchaser     3926 non-null object
paddress      5810 non-null object
rparty        5191 non-null object
raddress      5191 non-null object
attorney      5946 non-null object
descript      5121 non-null object
location      2845 non-null object
defend_att    619 non-null object
with          2470 non-null object
status        184 non-null object
ssold         5994 non-null int64
SHF           5994 non-null int64
LATITUDE      5981 non-null float64
LONGITUDE     5981 non-null float64
dtypes: dat

In [127]:
sa.head(2)

,parcel,address,salesdt,saleno,sold_amt,appraisal,minbid,scaseno,withdrawn,plaintiff,...,attorney,descript,location,defend_att,with,status,ssold,SHF,LATITUDE,LONGITUDE
0,121-32-012,10715 MT. CARMEL AVE A SINGLE FRAME DWELLING,2001-03-12,12,NaN,"$12,000","$8,000","392,847 ALIAS",BANKRUPTCY,"COUNTRYWIDE HOME LOANS, INC., FKA, ETC.",...,KRISS FELTY,NaN,CLEVELAND East of River,NaN,BANKR,NaN,0,1,41.491059,-81.610578
1,121-32-012,10715 MT CARMEL AVE.,2002-05-06,13,NaN,"$25,000","$16,667","392,847 PLURIES",NaN,"COUNTRYWIDE HOME LOANS, INC., ETC",...,KRISS FELTY,NaN,CLEVELAND East of River,NaN,NaN,NaN,0,1,41.491059,-81.610578


In [145]:
sa.loc[sa.appraisal.notnull(), 'appraisal']= sa.loc[sa.appraisal.notnull(), 'appraisal'].apply(parse_amount)

In [156]:
sa[sa['sdefend']=='MAURICE MELTON, ET AL']

,parcel,address,salesdt,saleno,sold_amt,appraisal,minbid,scaseno,withdrawn,plaintiff,...,attorney,descript,location,defend_att,with,status,ssold,SHF,LATITUDE,LONGITUDE
1,121-32-012,10715 MT CARMEL AVE.,2002-05-06,13,NaN,25000,"$16,667","392,847 PLURIES",NaN,"COUNTRYWIDE HOME LOANS, INC., ETC",...,KRISS FELTY,NaN,CLEVELAND East of River,NaN,NaN,NaN,0,1,41.491059,-81.610578
2,121-32-012,10715 CARMEL AVE,2002-11-25,16,"$16,667",25000,"$16,500","392,847 2ND PLURIES",NaN,COUNTRYWIDE HOME LOANS ETC.,...,KRISS FELTY,NaN,CLEVELAND East of River,NaN,NaN,NaN,1,1,41.491059,-81.610578
5,121-32-014,10715 MT CARMEL AVE.,2002-05-06,13,NaN,25000,"$16,667","392,847 PLURIES",NaN,"COUNTRYWIDE HOME LOANS, INC., ETC",...,KRISS FELTY,NaN,CLEVELAND East of River,NaN,NaN,NaN,0,1,41.491219,-81.610437
6,121-32-014,10715 CARMEL AVE,2002-11-25,16,"$16,667",25000,"$16,500","392,847 2ND PLURIES",NaN,COUNTRYWIDE HOME LOANS ETC.,...,KRISS FELTY,NaN,CLEVELAND East of River,NaN,NaN,NaN,1,1,41.491219,-81.610437


In [169]:
tem = sa.loc[sa.salesdt>dt.datetime(2011, 3, 1),['parcel','ssold']].groupby('parcel').sum()
tem.columns = ['auction']

tci = pd.merge(tci, tem, \
             how='left', left_on='parcel', right_index=True)

In [170]:
tci[tci.auction.isnull()] = 0
tci[tci.auction > 0] = 1

## Tax bill

In [171]:
tb = pd.read_csv('clean_data/taxbill_tci.csv')

In [172]:
tb.head()

,PROPERTY_NUMBER,LENDER_PROCESS_TYPE,GRAND_TOTAL_BALANCE,TOTAL_NET_DELQ_BALANCE,TOTAL_NET_BALANCE_OWED_1ST,TOTAL_NET_BALANCE_OWED_2ND,GRAND_TOTAL_BALANCE.1,FORECLOSURE_FLAG,MAIL_STREET_NUMBER,MAIL_STREET_NAME,LOCATION_STREET_NO,LOCATION_STREET_NAME,HOMESTEAD_FLAG
0,129-23-013,0,113.04,0.00,0.00,113.04,113.04,NaN,2999,PAYNE,12008,Forest,NaN
1,126-16-034,2500,655.43,0.00,0.00,655.43,655.43,NaN,2999,PAYNE,10005,Cumberland,NaN
2,121-31-003,NaN,28.99,0.00,0.00,28.99,28.99,NaN,2417,Woodhill,2427,Woodhill,NaN
3,121-31-008,0,6298.02,5169.85,577.85,550.32,6298.02,1,NaN,Ingersoll,NaN,Ingersoll,NaN
4,121-32-124,1501,308.39,0.00,0.00,308.39,308.39,NaN,2399,Mapleside,2399,Mapleside,NaN


In [174]:
len(set(tb.PROPERTY_NUMBER))

6075